In [3]:
import nltk
#nltk.download('stopwords')

In [7]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [8]:

#!pip install pyLDAvis

In [9]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
d=pd.read_csv("preprocessed.csv")

In [11]:
d.head(4)

,created_at,hashtags,id,lang,retweet_count,source,text,user_location,user_verified,cleaned_tweet,Subjectivity,Polarity,Sentiment
0,2022-02-28 07:22:45,NaN,1498196962780213255,en,0.0,Android,@newsbusters It's because Trump is the head of...,USA,True,newsbusters trump head republican party idiot ...,0.800000,-0.800,Negative
1,2022-02-28 07:22:45,NaN,1498196962914750468,en,4642.0,iPhone,Ukraine Russia conflict highlights that geogra...,India,True,ukraine russia conflict highlight geography ma...,0.497222,0.125,Positive
2,2022-02-28 07:22:45,NaN,1498196963174453248,en,0.0,WebApp,@Puno_de_hierro so to kill handful of people i...,CA,True,punodehierro kill handful people worth invadin...,0.050000,0.050,Positive
3,2022-02-28 07:22:45,NaN,1498196959735463938,en,376.0,Android,"Union ministers Hardeep Puri,Jyotiraditya Scin...",India,True,union minister hardeep purijyotiraditya scindi...,0.000000,0.000,Neutral


In [12]:
df=d

# Tweet Preprocessing

In [13]:
# import emoji
# #remove emoji
# def cleantweet(text):
#     text=emoji.get_emoji_regexp().sub(u'', text)
#     return text
# df['text']=df["text"].apply(cleantweet)

In [14]:
df["cleaned_tweet"]=df["cleaned_tweet"].astype("str")

In [15]:
data = df.cleaned_tweet.values.tolist()

In [16]:
data

['newsbusters trump head republican party idiot praise putin invaded sovereign nation',
 'ukraine russia conflict highlight geography matter india china one side pakistan nuclear enabled dependence russian military equipment u far away challenge one thing war teach sure atmanirbhar',
 'punodehierro kill handful people worth invading destroying million ukrainian life',
 'union minister hardeep purijyotiraditya scindia kiren rijijuv k singh go neighbouring country ukraine help studentssources',
 'breaking president putin order nuclear deterrent force high alert international tension build russia invasion ukraine',
 'moscow underground sometimes act resistance doesn’t loud bold ukraine slavaukraini',
 'ukrainian bank made simple page english want help ukrainian army war russiawe accept payment card bank worldglory ukrainelink',
 'va hilo situación actual kievukraine ‼️⚡el alcalde de kiev🇺🇦 dijo que la capital ucraniana ahora está rodeada por los rusos🇷🇺 que la evacuación de los residentes

In [17]:
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

In [18]:
data

['newsbusters trump head republican party idiot praise putin invaded sovereign nation',
 'ukraine russia conflict highlight geography matter india china one side pakistan nuclear enabled dependence russian military equipment u far away challenge one thing war teach sure atmanirbhar',
 'punodehierro kill handful people worth invading destroying million ukrainian life',
 'union minister hardeep purijyotiraditya scindia kiren rijijuv k singh go neighbouring country ukraine help studentssources',
 'breaking president putin order nuclear deterrent force high alert international tension build russia invasion ukraine',
 'moscow underground sometimes act resistance doesn’t loud bold ukraine slavaukraini',
 'ukrainian bank made simple page english want help ukrainian army war russiawe accept payment card bank worldglory ukrainelink',
 'va hilo situación actual kievukraine ‼️⚡el alcalde de kiev🇺🇦 dijo que la capital ucraniana ahora está rodeada por los rusos🇷🇺 que la evacuación de los residentes

In [19]:
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

In [20]:
data

['newsbusters trump head republican party idiot praise putin invaded sovereign nation',
 'ukraine russia conflict highlight geography matter india china one side pakistan nuclear enabled dependence russian military equipment u far away challenge one thing war teach sure atmanirbhar',
 'punodehierro kill handful people worth invading destroying million ukrainian life',
 'union minister hardeep purijyotiraditya scindia kiren rijijuv k singh go neighbouring country ukraine help studentssources',
 'breaking president putin order nuclear deterrent force high alert international tension build russia invasion ukraine',
 'moscow underground sometimes act resistance doesn’t loud bold ukraine slavaukraini',
 'ukrainian bank made simple page english want help ukrainian army war russiawe accept payment card bank worldglory ukrainelink',
 'va hilo situación actual kievukraine ‼️⚡el alcalde de kiev🇺🇦 dijo que la capital ucraniana ahora está rodeada por los rusos🇷🇺 que la evacuación de los residentes

In [21]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

In [22]:
data

['newsbusters trump head republican party idiot praise putin invaded sovereign nation',
 'ukraine russia conflict highlight geography matter india china one side pakistan nuclear enabled dependence russian military equipment u far away challenge one thing war teach sure atmanirbhar',
 'punodehierro kill handful people worth invading destroying million ukrainian life',
 'union minister hardeep purijyotiraditya scindia kiren rijijuv k singh go neighbouring country ukraine help studentssources',
 'breaking president putin order nuclear deterrent force high alert international tension build russia invasion ukraine',
 'moscow underground sometimes act resistance doesn’t loud bold ukraine slavaukraini',
 'ukrainian bank made simple page english want help ukrainian army war russiawe accept payment card bank worldglory ukrainelink',
 'va hilo situación actual kievukraine ‼️⚡el alcalde de kiev🇺🇦 dijo que la capital ucraniana ahora está rodeada por los rusos🇷🇺 que la evacuación de los residentes

In [23]:
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [24]:
data

['newsbusters trump head republican party idiot praise putin invaded sovereign nation',
 'ukraine russia conflict highlight geography matter india china one side pakistan nuclear enabled dependence russian military equipment u far away challenge one thing war teach sure atmanirbhar',
 'punodehierro kill handful people worth invading destroying million ukrainian life',
 'union minister hardeep purijyotiraditya scindia kiren rijijuv k singh go neighbouring country ukraine help studentssources',
 'breaking president putin order nuclear deterrent force high alert international tension build russia invasion ukraine',
 'moscow underground sometimes act resistance doesn’t loud bold ukraine slavaukraini',
 'ukrainian bank made simple page english want help ukrainian army war russiawe accept payment card bank worldglory ukrainelink',
 'va hilo situación actual kievukraine ‼️⚡el alcalde de kiev🇺🇦 dijo que la capital ucraniana ahora está rodeada por los rusos🇷🇺 que la evacuación de los residentes

In [25]:
print(data[:1])

['newsbusters trump head republican party idiot praise putin invaded sovereign nation']


In [26]:
#link remove
data = [re.sub(r'https?:\/\/\S+','',sent) for sent in data]

In [27]:
data

['newsbusters trump head republican party idiot praise putin invaded sovereign nation',
 'ukraine russia conflict highlight geography matter india china one side pakistan nuclear enabled dependence russian military equipment u far away challenge one thing war teach sure atmanirbhar',
 'punodehierro kill handful people worth invading destroying million ukrainian life',
 'union minister hardeep purijyotiraditya scindia kiren rijijuv k singh go neighbouring country ukraine help studentssources',
 'breaking president putin order nuclear deterrent force high alert international tension build russia invasion ukraine',
 'moscow underground sometimes act resistance doesn’t loud bold ukraine slavaukraini',
 'ukrainian bank made simple page english want help ukrainian army war russiawe accept payment card bank worldglory ukrainelink',
 'va hilo situación actual kievukraine ‼️⚡el alcalde de kiev🇺🇦 dijo que la capital ucraniana ahora está rodeada por los rusos🇷🇺 que la evacuación de los residentes

In [28]:
data=[re.sub(r'#','',sent)for sent in data]

In [29]:
data

['newsbusters trump head republican party idiot praise putin invaded sovereign nation',
 'ukraine russia conflict highlight geography matter india china one side pakistan nuclear enabled dependence russian military equipment u far away challenge one thing war teach sure atmanirbhar',
 'punodehierro kill handful people worth invading destroying million ukrainian life',
 'union minister hardeep purijyotiraditya scindia kiren rijijuv k singh go neighbouring country ukraine help studentssources',
 'breaking president putin order nuclear deterrent force high alert international tension build russia invasion ukraine',
 'moscow underground sometimes act resistance doesn’t loud bold ukraine slavaukraini',
 'ukrainian bank made simple page english want help ukrainian army war russiawe accept payment card bank worldglory ukrainelink',
 'va hilo situación actual kievukraine ‼️⚡el alcalde de kiev🇺🇦 dijo que la capital ucraniana ahora está rodeada por los rusos🇷🇺 que la evacuación de los residentes

In [30]:
data=[re.sub(r'RT[\s]+','',sent)for sent in data]

In [31]:
data

['newsbusters trump head republican party idiot praise putin invaded sovereign nation',
 'ukraine russia conflict highlight geography matter india china one side pakistan nuclear enabled dependence russian military equipment u far away challenge one thing war teach sure atmanirbhar',
 'punodehierro kill handful people worth invading destroying million ukrainian life',
 'union minister hardeep purijyotiraditya scindia kiren rijijuv k singh go neighbouring country ukraine help studentssources',
 'breaking president putin order nuclear deterrent force high alert international tension build russia invasion ukraine',
 'moscow underground sometimes act resistance doesn’t loud bold ukraine slavaukraini',
 'ukrainian bank made simple page english want help ukrainian army war russiawe accept payment card bank worldglory ukrainelink',
 'va hilo situación actual kievukraine ‼️⚡el alcalde de kiev🇺🇦 dijo que la capital ucraniana ahora está rodeada por los rusos🇷🇺 que la evacuación de los residentes

In [32]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['newsbusters', 'trump', 'head', 'republican', 'party', 'idiot', 'praise', 'putin', 'invaded', 'sovereign', 'nation']]


In [33]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['newsbusters', 'trump', 'head', 'republican_party', 'idiot', 'praise', 'putin', 'invaded', 'sovereign', 'nation']


In [34]:
import requests
requests.get('https://google.com', verify=False)
import spacy
from spacy.cli.download import download
#download(model="en_core_web_sm")
spacy.load('en_core_web_sm')
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [35]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['newsbuster', 'head', 'praise', 'invade', 'sovereign', 'nation']]


# Creating Dictionary , Corpus 

In [36]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [37]:
id2word[0]


'head'

In [38]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('head', 1),
  ('invade', 1),
  ('nation', 1),
  ('newsbuster', 1),
  ('praise', 1),
  ('sovereign', 1)]]

# LDA Modelling

In [39]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [40]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.050*"say" + 0.019*"know" + 0.014*"come" + 0.011*"kill" + 0.011*"good" + '
  '0.010*"soldier" + 0.009*"force" + 0.009*"continue" + 0.008*"destroy" + '
  '0.008*"show"'),
 (1,
  '0.064*"russian" + 0.018*"go" + 0.018*"people" + 0.013*"call" + 0.012*"take" '
  '+ 0.011*"military" + 0.011*"trump" + 0.011*"get" + 0.010*"country" + '
  '0.009*"stop"'),
 (2,
  '0.042*"ukrainian" + 0.019*"make" + 0.014*"support" + 0.013*"need" + '
  '0.012*"still" + 0.011*"world" + 0.011*"also" + 0.011*"troop" + 0.010*"die" '
  '+ 0.010*"fight"'),
 (3,
  '0.025*"would" + 0.024*"war" + 0.016*"help" + 0.011*"tell" + 0.011*"today" + '
  '0.010*"report" + 0.009*"new" + 0.009*"could" + 0.009*"must" + 0.008*"day"')]


In [41]:
# # Compute Perplexity
# print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# # Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

In [42]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model, corpus, id2word)
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [43]:
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.298900  0.002506       1        1  35.073574
2     -0.103669  0.230949       2        1  22.519333
3     -0.099986 -0.202053       3        1  21.962468
0     -0.095244 -0.031402       4        1  20.444626, topic_info=          Term         Freq        Total Category  logprob  loglift
135    russian  2017.000000  2017.000000  Default  30.0000  30.0000
498        say   910.000000   910.000000  Default  29.0000  29.0000
21   ukrainian   842.000000   842.000000  Default  28.0000  28.0000
362      would   489.000000   489.000000  Default  27.0000  27.0000
15         war   470.000000   470.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
918    include    86.482659    87.190638   Topic4  -5.3568   1.5793
496    believe    84.593805    85.290563   Topic4  -5.3789   1.5792
12        sure    88.098281    88.836961   Topic4  -5.3383   1.5791
10         far    84.495916    85.207473   Topic4  -5.3801   1.5791
682        put    84.800454    85.515577   Topic4  -5.3765   1.5791

[153 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
252       2  0.997070    also
84        2  0.986972  always
155       4  0.989977     amp
1292      2  0.993938    area
127       1  0.970451     ask
...     ...       ...     ...
1597      3  0.996161    word
461       1  0.997022    work
286       2  0.997971   world
362       3  0.997578   would
223       2  0.995847    year

[125 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 4, 1])

In [44]:
df.shape

(12397, 13)